<a href="https://colab.research.google.com/github/ibmkeiba/1_WebScraper/blob/main/colab/netkeiba4/netkeiba_4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **【netkeiba.com】 本番 Stage4 #1**

> **今月分のレース一覧とレース情報/結果を自動的に取得**

* urllib, selenium(chromium) でアクセス
* BeaurifulSoupで解析
* レースカレンダーからレースがある日を取得 

    【取得するレース開催情報】
        開催年月, 開催日

> **参考URL**

* 【netkeiba.com】トップページ (https://www.netkeiba.com/)  
* 【netkeiba.com】開催一覧　(https://race.netkeiba.com/top/calendar.html)  
* 【netkeiba.com】開催一覧 (年月指定) (https://race.netkeiba.com/top/calendar.html?year=2021&month=8)
* 【netkeiba.com】開催レース一覧（特定日） (https://race.netkeiba.com/top/race_list.html?kaisai_date=20210801)

## **初期設定**

### **ライブラリインポート**

In [1]:
# choromiumとseleniumをインストール
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://sec

In [3]:
# ライブラリのインポート
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

In [4]:
# ライブラリのインポート
from urllib import request as req
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import time
import shutil

## **ユーザー関数**

### **①レース開催日の取得**

In [25]:
# URLからソースコードを取得する関数 <urllib>  【url > soup】
def my_request(url):

    # URLを開く
    res = req.urlopen(url)

    # URLのソースコードを取得
    soup = BeautifulSoup(res, 'html.parser')

    # 暗黙的待機（サーバーの負荷軽減目的）
    time.sleep(0.1)
    
    return soup

In [26]:
# 開催一覧から選択中の年月を返す関数 【soup > 表示している開催一覧ページの年月（文字列型）】

def my_selected_ym(soup):

    # 選択中の年 > 例 2021
    selected_y = soup.select_one('select#cal_select_year option[selected@selected]').text

    # 選択中の月 > 例 8
    selected_m = soup.select_one('div.RaceNumWrap.CalendarMonth ul li.Active a').text.replace('月', '')

    # 選択中の年月の6桁文字表記 例 202109
    selected_ym = selected_y + selected_m.zfill(2)

    return selected_ym

In [27]:
# 開催一覧のURLから現在の年月と開催がある日を取得 【url > ym（文字列型）, days（整数のリスト型）】
def my_get_raceday_from_url(url_calendar='https://race.netkeiba.com/top/calendar.html'):

    # URLのソースコードを取得
    soup = my_request(url_calendar)

    # 表示中の年月を取得
    ym = my_selected_ym(soup)

    #レース開催がある日のエレメント（なければ []）
    race_day_elements = soup.select('table.Calendar_Table td.RaceCellBox a')

    # レース開催がある日を回す
    days = []
    for ele in race_day_elements:
        
        # レース開催日 例 5
        day = int(ele.select_one('span.Day').text)

        # リストに追加
        days.append(day)

    return ym, days

### **②開催レース一覧**

In [30]:
# ドライバーを起動 【 - > driver 】
def my_start_driver():

    # オプション設定（★　Colabでそのまま使用できる設定 ★）
    options = webdriver.ChromeOptions()
    options.add_argument('--headless') # Colabではヘッドレス必須*
    options.add_argument('--no-sandbox') # Colab環境でのヘッドレスで必須*
    options.add_argument('--disable-gpu') # soup.prettify() の実行に必要
    options.add_argument('--disable-dev-shm-usage') # メモリ不足回避に有効

    # ドライバーを起動
    driver = webdriver.Chrome('chromedriver', options=options)

    # 要素が見つかるまでの待機時間(秒)を設定
    driver.implicitly_wait(5)

    return driver

In [31]:
# 指定URLのソースコードを取得 【 url > soup 】
def my_get_source_driver(url):

    global driver

    # もしグローバル変数にdriverがなければ起動
    try:
        driver.current_url
    except:
        driver = my_start_driver()
        print('■ driverを起動しました\n')

    # ブラウザでURLにアクセス
    driver.get(url)

    # ソースコードをBeautifulSopuに変換
    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')

    return soup

In [33]:
# 目的のレースのエレメントから情報を取得
def my_race_info(race_element):

    # レースURL 'https://', race_url
    race_url = race_element.select_one('a.').attrs['href'].replace('..', 'https://race.netkeiba.com')

    # レースID
    race_id = race_url.split('?race_id=')[1].split('&')[0]

    # レース番 '1R', r
    r = race_element.select_one('a. div.Race_Num span').text

    # レースタイトル '3歳未勝利', title
    title = race_element.select_one('a. span.ItemTitle').text

    # 開始時刻 '10:10 ', time
    start_time = race_element.select_one('a. span.RaceList_Itemtime').text

    # コースタイプと距離 'ダ1200m', distance
    #distance = race_element.select_one('a. span.RaceList_ItemLong').text
    
    try:
        distance = race_element.select_one('a. span.RaceList_ItemLong').text

    except AttributeError:
        distance = race_element.select('a. div.RaceData span')[1].text
    
    except:
        print('◆ Select error, ', 'distance ↓')
        distance = ''

    # 出場馬数 '16頭', number
    number = race_element.select_one('a. span.RaceList_Itemnumber').text.replace(' ','')

    # レースグレード, '3', grade
    try:
        grade = race_element.select_one('a. span.Icon_GradeType').attrs['class']
    except: # 存在しない場合
        grade = ''
    else: # 存在する場合
        grade = grade[1].replace('Icon_GradeType', '')

    return [r, race_id, start_time, title, grade, number, distance, race_url]

In [34]:
# ②開催レース一覧のソースコードから全レースの情報を取得 【 soup > DataFrame 】
def my_race_list(soup):

    # 1日分のレース情報を格納するリスト
    races_list = []

    # 開催年月日（ymd） 文字型
    ymd = soup.select_one('ul#date_list_sub li.Active').attrs['date']

    # 開催日（曜日）
    date = soup.select_one('ul#date_list_sub li.Active a').attrs['title']

    # 開催場所のエレメントリスト > 3箇所
    place_elelments = soup.select('div.RaceList_Box > dl.RaceList_DataList')

    # 開催場所を回す
    for i, place_ele in enumerate(place_elelments):

        # 開催場所
        place = place_ele.select_one('p.RaceList_DataTitle').text.split(' ')[1]

        # レース一覧のエレメントリスト
        race_elements = place_ele.select('dd.RaceList_Data ul li')

        # レース会場名とレース数を表示
        print('・{}年{}　> {} 【{}】 {} レース'.format(ymd[:4], date, i+1, place, len(race_elements)))

        # レースを回す
        for j, race_ele in enumerate(race_elements):

            # レースのエレメントから情報を取得
            race_list = my_race_info(race_ele)
            # レース情報を表示
            print(race_list)

            # 1日分のレース情報に追加
            races_list.append([ymd, date, place] + race_list)
            
    # 1日分のレース情報をDataFrame形式に変換
    ymd_df = pd.DataFrame(races_list, index=None, columns=['ymd', 'date', 'place', 'r', 'race_id', 'start_time', 'title', 'grade', 'number', 'distance', 'url'])

    return ymd_df

## **メインプログラム**

### **①レース開催日の取得**

In [38]:
# 指定年月URLからレース開催がある日とそのURLのリストを取得
ym, days = my_get_raceday_from_url()
print(ym, days)

202109 [4, 5, 11, 12, 18, 19, 20, 25, 26]


### **②開催レース一覧**

In [36]:
# 全レース情報を格納する空のDataFrameを作成
races_df = pd.DataFrame()

# 日を回す d[1:index, 2:d]
for d in days:

    # レース開催日のURLを作成
    url_ymd = 'https://race.netkeiba.com/top/race_list.html?kaisai_date=' + str(ym) + str(d).zfill(2)
    print('\n\n①', str(ym) + str(d).zfill(2), url_ymd, '\n')

    # 全ての会場とレースの情報を取得

    # URL（開催日）のソースコードを取得（selenium）
    soup = my_get_source_driver(url_ymd)

    # 特定日の全レース情報を取得
    race_df = my_race_list(soup)

    # 全日の全レース情報を格納
    races_df = races_df.append(race_df).reset_index(drop=True)



① 20210904 https://race.netkeiba.com/top/race_list.html?kaisai_date=20210904 

・2021年9月4日(土)　> 1 【新潟】 12 レース
['1R', '202104040701', '10:10 ', '3歳以上障害未勝利', '', '13頭', '障2850m', 'https://race.netkeiba.com/race/result.html?race_id=202104040701&rf=race_list']
['2R', '202104040702', '10:45 ', '2歳未勝利', '', '15頭', 'ダ1800m', 'https://race.netkeiba.com/race/result.html?race_id=202104040702&rf=race_list']
['3R', '202104040703', '11:15 ', '3歳未勝利', '', '15頭', 'ダ1800m', 'https://race.netkeiba.com/race/result.html?race_id=202104040703&rf=race_list']
['4R', '202104040704', '11:45 ', '3歳以上障害OP', '5', '11頭', '障3250m', 'https://race.netkeiba.com/race/result.html?race_id=202104040704&rf=race_list']
['5R', '202104040705', '12:35 ', '2歳新馬', '', '10頭', '芝2000m', 'https://race.netkeiba.com/race/result.html?race_id=202104040705&rf=race_list']
['6R', '202104040706', '13:05 ', '3歳未勝利', '', '18頭', '芝1600m', 'https://race.netkeiba.com/race/result.html?race_id=202104040706&rf=race_list']
['7R', '202104040707', '

In [37]:
races_df

,ymd,date,place,r,race_id,start_time,title,grade,number,distance,url
0,20210904,9月4日(土),新潟,1R,202104040701,10:10,3歳以上障害未勝利,,13頭,障2850m,https://race.netkeiba.com/race/result.html?rac...
1,20210904,9月4日(土),新潟,2R,202104040702,10:45,2歳未勝利,,15頭,ダ1800m,https://race.netkeiba.com/race/result.html?rac...
2,20210904,9月4日(土),新潟,3R,202104040703,11:15,3歳未勝利,,15頭,ダ1800m,https://race.netkeiba.com/race/result.html?rac...
3,20210904,9月4日(土),新潟,4R,202104040704,11:45,3歳以上障害OP,5,11頭,障3250m,https://race.netkeiba.com/race/result.html?rac...
4,20210904,9月4日(土),新潟,5R,202104040705,12:35,2歳新馬,,10頭,芝2000m,https://race.netkeiba.com/race/result.html?rac...
...,...,...,...,...,...,...,...,...,...,...,...
235,20210926,9月26日(日),中京,8R,202107050708,13:50,3歳以上1勝クラス,,10頭,芝2000m,https://race.netkeiba.com/race/result.html?rac...
236,20210926,9月26日(日),中京,9R,202107050709,14:25,知多特別,17,18頭,芝1200m,https://race.netkeiba.com/race/result.html?rac...
237,20210926,9月26日(日),中京,10R,202107050710,15:01,桶狭間S,16,16頭,ダ1400m,https://race.netkeiba.com/race/result.html?rac...
238,20210926,9月26日(日),中京,11R,202107050711,15:35,神戸新聞杯,2,10頭,芝2200m,https://race.netkeiba.com/race/result.html?rac...
